In [ ]:
#export
from nb_006a import *

# Camvid

## Setup

In [ ]:
PATH = Path('data/camvid')
PATH_X = PATH/'701_StillsRaw_full'
PATH_Y = PATH/'LabeledApproved_full'
PATH_Y_PROCESSED = PATH/'LabelProcessed'
label_csv = PATH/'label_colors.txt'

PATH_Y_PROCESSED.mkdir(exist_ok=True)

In [ ]:
list(PATH_Y.iterdir())[0]

In [ ]:
def parse_code(l):
    a,b = [c for c in l.strip().split("\t") if c]
    return tuple(int(o) for o in a.split(' ')), b
label_codes,label_names = zip(*[parse_code(l) for l in open(PATH/"label_colors.txt")])
label_codes,label_names = list(label_codes),list(label_names)
name2code = dict(zip(label_names, label_codes))
void_code = name2code['Void']
void_id = 0
code2id = {code:(i+1) for i,code in enumerate(label_codes) if code != void_code }
code2id[void_code] = void_id
id2code = {v:k for k,v in code2id.items()}
n_labels = len(label_codes)

In [ ]:
def get_y_fn(x_fn): return PATH_Y/f'{x_fn.name[:-4]}_L.png'
def get_y_proc_fn(y_fn): return PATH_Y_PROCESSED/f'{y_fn.name[:-6]}_P.png'

In [ ]:
x_fns = [o for o in PATH_X.iterdir() if o.is_file()]
y_fns = [get_y_fn(o) for o in x_fns]
y_proc_fns = [get_y_proc_fn(o) for o in y_fns]

In [ ]:
def colors_to_codes(color_data):
    h, w = color_data.shape[1:3]
    data = color_data.view(3, -1)
    n_pixels = data.shape[1]
    proc_data = np.zeros((1, n_pixels),dtype=np.uint8)
    for i in range(n_pixels):
        proc_data[:,i] = code2id.get(tuple(data[:,i].numpy()), 0)
    proc_data.resize((1, h, w))
    return proc_data

def codes_to_colors(label_data):
    h, w = label_data.shape[1:3]
    data = label_data.flatten()
    n_pixels = data.shape[0]
    proc_data = np.zeros((3, n_pixels),dtype=np.float)
    for i in range(n_pixels):
        proc_data[:,i] = id2code.get(data[i], (0,0,0))
    proc_data.resize((3, h, w))
    return proc_data
    
def process_file(fns):
    yfn, pfn = fns
    if not pfn.exists():
        y_data = open_mask(yfn).px.long()
        proc_data = colors_to_codes(y_data)
        img = PIL.Image.fromarray(proc_data[0])
        img.save(pfn)
    return pfn

from concurrent.futures import ProcessPoolExecutor
def process_label_files(y_fns, y_proc_fns):
    ex = ProcessPoolExecutor(16)
    for pfn in ex.map(process_file, zip(y_fns, y_proc_fns)):
        pass

def codes_to_image(codes):
    return Image(tensor(codes_to_colors(codes)).int())

In [ ]:
# i = 0
# x_img = open_image(x_fns[i])
# y_img_mask = open_mask(y_fns[i])
# y_img = Image(y_img_mask.data.int())
# y_code = colors_to_codes(y_img.px)
# y_img2 = Image(tensor(codes_to_colors(y_code)).int())
# y_img.show(), y_img2.show()

In [ ]:
%time process_label_files(y_fns, y_proc_fns)

In [ ]:
def get_datasets(path, valid_pct=0.2):
    x_fns = [o for o in path.iterdir() if o.is_file()]
    y_fns = [get_y_fn(o) for o in x_fns]
    y_proc_fns = [get_y_proc_fn(o) for o in y_fns]
    total = len(x_fns)
    
    train, valid = random_split(valid_pct, x_fns, y_proc_fns)
    return (MatchedFilesDataset(*train),
            MatchedFilesDataset(*valid))

In [ ]:
def get_tfm_datasets(size):
    datasets = get_datasets(PATH_X)
    tfms = get_transforms(do_flip=True, max_rotate=4, max_lighting=0.2)
    return transform_datasets(*datasets, tfms=tfms, tfm_y=True, size=size)

In [ ]:
default_norm,default_denorm = normalize_funcs(*imagenet_stats)
bs = 8
size = 512

In [ ]:
tfms = get_transforms(do_flip=True, max_rotate=4, max_lighting=0.2)

In [ ]:
def get_data(size, bs):
    return DataBunch.create(*get_tfm_datasets(size), bs=bs, tfms=default_norm)

In [ ]:
data = get_data(size, bs)

In [ ]:
x, y = data.train_ds[0]
x.shape, y.shape, y.data.dtype

## Unet

In [ ]:
def accuracy_no_void(input, target):
    mask = target != void_id
    return accuracy(input[mask], target[mask])

metrics=[]
lr = 1e-3

In [ ]:
# cross_entropy loss expects target to be long type
def my_loss(pred, target):
    return F.cross_entropy(pred, target.squeeze().long())
    
body = create_body(tvm.resnet34(True), 2)
model = DynamicUnet(body, n_classes=len(label_codes)).cuda()
learn = Learner(data, model, metrics=[],
                loss_fn=my_loss)
learn.split([model[0][6], model[1]])
learn.freeze()

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
lr = 1e-2

In [ ]:
learn.fit_one_cycle(1, slice(lr), pct_start=0.05)

In [ ]:
learn.fit_one_cycle(6, slice(lr), pct_start=0.05)

In [ ]:
learn.save('u0')

In [ ]:
learn.load('u0')
x,y = next(iter(learn.data.valid_dl))
py = learn.model(x).detach()
py = py.softmax(dim=1).max(dim=1, keepdim=True)[1]
x,y,py = x.cpu(),y.cpu(),py.cpu()
x = default_denorm(x)

In [ ]:
n = 4
fig, axs = plt.subplots(n,3,figsize=(10,10), sharey=True)
for i in range(n):
    Image(x[i]).show(ax=axs[i][0])
    codes_to_image(y[i].numpy()).show(ax=axs[i][1])
    codes_to_image(py[i].numpy()).show(ax=axs[i][2])

In [ ]:
learn.unfreeze()
lr=1e-2

In [ ]:
learn.fit_one_cycle(6, slice(lr/100,lr), pct_start=0.05)

In [ ]:
size=640
bs = 4
learn.data = get_data(size, bs)

In [ ]:
#learn.freeze()

In [ ]:
learn.fit_one_cycle(6, slice(lr), pct_start=0.05)

In [ ]:
id2code

## Fin